In [69]:
#Dependencies
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
combined_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

combined_data.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [70]:
#Calculate the total number of schools
total_schools = school_data['school_name'].count()
total_schools

#district student count
dist_student_count = school_data['size'].sum()

#Calculate the total number of students
total_students = student_data['Student ID'].count()
total_students

#student count from student file (to verify with district student count)
total_student_rec = student_data['student_name'].count()

#Calculate the total budget
total_budget = school_data['budget'].sum()
total_budget 


#calculations for number and % passing reading
num_passing_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

#calculations for number and % passing math
num_passing_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec

#average math score calculation
avg_math_score = student_data['math_score'].mean()

#average reading score calculation
avg_reading_score = student_data['reading_score'].mean()

#Overall Passing Rate Calculations
overall_pass = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]['student_name'].count()/total_student_rec

# district dataframe from dictionary

district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading":[perc_pass_reading*100],
    "% Passing Math": [perc_pass_math*100],
    "% Overall Passing": [overall_pass*100],
})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             '% Overall Passing']]

#format cells
dist_sum.style.format({ "Total Students":"{:,}", "Total Budget": "${:,.2f}", 
                       "Average Math Score": "{:.6f}", 
                       "Average Reading Score": "{:.6f}", 
                       "% Passing Math": "{:.6}", 
                       "% Passing Reading": "{:.6}", 
                       "% Overall Passing": "{:.6}"})



,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing
0,15,"39,170","$24,649,428.00",81.877840,78.985371,85.8055,74.9809,65.1723


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [71]:
# Group by school
#
school_group = combined_data.set_index('school_name').groupby(['school_name'])

# School type
#
school_type = school_data.set_index('school_name')['type']

# Total students by school
students_per_school = school_group['Student ID'].count()

# Total school budget
school_budget = school_data.set_index('school_name')['budget']

# Per student budget
#
per_student_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

# Average scores on Math and Reading in school_group
#
average_math_score = school_group['math_score'].mean()
average_read_score = school_group['reading_score'].mean()

# 
# Calculation for Percentage (%) of students that passed math, reading
# and % overall passing based on commbined data by schools and students
#
students_passing_math = combined_data[combined_data['math_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school
students_passing_reading = combined_data[combined_data['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school 
overall_passing_students = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/students_per_school

# Calcuate % for each 
students_passing_math = students_passing_math*100
students_passing_reading = students_passing_reading*100
overall_passing_students = overall_passing_students*100

# Creating dataframe to hold the above results to be displayed further 
#
school_summary_dataframe = pd.DataFrame({
    "School Type": school_type,
    "Total Students": students_per_school,
    "Per Student Budget": per_student_budget,
    "Total School Budget": school_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_read_score,
    '% Passing Math': students_passing_math,
    '% Passing Reading': students_passing_reading,
    '% Overall Passing': overall_passing_students
})


#
# Formatting the dataframe as per the the given example 
#
school_summary_dataframe.style.format({'Total Students': '{:}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.6f}", 
                          'Average Reading Score': "{:.6f}", 
                          "% Passing Math": "{:.6}", 
                          "% Passing Reading": "{:.6}", 
                          "% Overall Passing": "{:.6}"})




,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,$628,"$3,124,928",77.048432,81.033963,66.6801,81.9333,54.6423
Cabrera High School,Charter,1858,$582,"$1,081,356",83.061895,83.975780,94.1335,97.0398,91.3348
Figueroa High School,District,2949,$639,"$1,884,411",76.711767,81.158020,65.9885,80.7392,53.2045
Ford High School,District,2739,$644,"$1,763,916",77.102592,80.746258,68.3096,79.299,54.2899
Griffin High School,Charter,1468,$625,"$917,500",83.351499,83.816757,93.3924,97.139,90.5995
Hernandez High School,District,4635,$652,"$3,022,020",77.289752,80.934412,66.753,80.863,53.5275
Holden High School,Charter,427,$581,"$248,087",83.803279,83.814988,92.5059,96.2529,89.2272
Huang High School,District,2917,$655,"$1,910,635",76.629414,81.182722,65.6839,81.3164,53.5139
Johnson High School,District,4761,$650,"$3,094,650",77.072464,80.966394,66.0576,81.2224,53.5392
Pena High School,Charter,962,$609,"$585,858",83.839917,84.044699,94.5946,95.9459,90.5405


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [72]:
top_five_performing_schools = school_summary_dataframe.sort_values("% Overall Passing", ascending=False)
top_five_performing_schools.head(5)


,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,582.0,1081356,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,638.0,1043130,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,625.0,917500,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,578.0,1319574,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,609.0,585858,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [73]:
bottom_five_performing_schools = school_summary_dataframe.sort_values("% Overall Passing", ascending=True)
bottom_five_performing_schools.head(5)


,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,637.0,2547363,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,639.0,1884411,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,655.0,1910635,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,652.0,3022020,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,650.0,3094650,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [74]:
#
# Create a pandas series for each grade for math_score and then group them by "school_name"
#
ninth_grade_math =  student_data[student_data["grade"] == "9th"].groupby("school_name").mean()["math_score"]
tenth_grade_math =  student_data[student_data["grade"] == "10th"].groupby("school_name").mean()["math_score"]
eleventh_grade_math = student_data[student_data["grade"] == "11th"].groupby("school_name").mean()["math_score"]
twelveth_grade_math =  student_data[student_data["grade"] == "12th"].groupby("school_name").mean()["math_score"]

#
# Create a dataframe to hold the data for the series
#
math_scores_by_grade_dataframe = pd.DataFrame({"9th":ninth_grade_math, "10th":tenth_grade_math, 
                                     "11th":eleventh_grade_math, "12th":twelveth_grade_math})            

#
# Formatting the datafram for display in clean form as per the given example
#

math_scores_by_grade = math_scores_by_grade_dataframe[["9th","10th","11th","12th"]].applymap("{:.6f}".format)

# Display the result data
#
math_scores_by_grade


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [75]:

#
# Create a pandas series for each grade for reading_score and then group them by "school_name"
#
ninth_grade_reading =  student_data[student_data["grade"] == "9th"].groupby("school_name").mean()["reading_score"]
tenth_grade_reading =  student_data[student_data["grade"] == "10th"].groupby("school_name").mean()["reading_score"]
eleventh_grade_reading = student_data[student_data["grade"] == "11th"].groupby("school_name").mean()["reading_score"]
twelveth_grade_reading =  student_data[student_data["grade"] == "12th"].groupby("school_name").mean()["reading_score"]

#
# Create a dataframe to hold the data for the series
#
reading_scores_by_grade_dataframe = pd.DataFrame({"9th":ninth_grade_reading, "10th":tenth_grade_reading, 
                                     "11th":eleventh_grade_reading, "12th":twelveth_grade_reading})            

#
# Formatting the datafram for display in clean form as per the given example
#

reading_scores_by_grade = reading_scores_by_grade_dataframe[["9th","10th","11th","12th"]].applymap("{:.6f}".format)

# Display the result data
#
reading_scores_by_grade



,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [76]:
# Creating speding bins
#
spending_bins = [0, 584, 629, 644, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

# slicing data based on "spending_bins" and "group_names"
#
combined_data['spending_bins'] = pd.cut(combined_data['budget']/combined_data['size'], bins, labels = group_names)

# Using groupby() based on new "spending_bins" just created 
#
spending_group = combined_data.groupby('spending_bins')

#
# Calculation of: Average Math, Reading scores
#
average_math_scores = spending_group['math_score'].mean()
average_reading_scores = spending_group['reading_score'].mean()

# 
# Calcuate passing rate of students paasing in math and reading 
# by spendings in various schools based on combined data
#  Using conditional statements
#
# Students paasing math
total_students_passing_in_math = combined_data[combined_data['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()
rate_of_students_passing_math = (total_students_passing_in_math)/spending_group['Student ID'].count()
percent_students_passing_math = rate_of_students_passing_math*100

# Students passing reading
total_students_passing_in_reading = combined_data[combined_data['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()
rate_of_students_passing_math = (total_students_passing_in_reading)/by_spending['Student ID'].count()
percent_students_passing_reading = rate_of_students_passing_math*100

# Overall studnets passing both the subjects
#
total_students_overall_passing = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()
overall_students_passing_rate = (total_students_overall_passing)/by_spending['Student ID'].count()
percent_overall_students_passing = overall_students_passing_rate*100
            
# Creating dataframe to hold the data on the above of students passing 
# based on spendings 
#
scores_by_school_spending = pd.DataFrame({
    "Average Math Score": average_math_scores,
    "Average Reading Score": average_reading_scores,
    '% Passing Math': percent_students_passing_math,
    '% Passing Reading': percent_students_passing_reading,
    '% Overall Passing': percent_overall_students_passing
            
})
            
# 
# Format column names based on the table given
#
scores_by_school_spending = scores_by_school_spending[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]

# Format dataframe to be displayed as per the given table
#
scores_by_school_spending.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              '% Overall Passing': '{:.2f}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
spending_bins,,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,83.53,83.84,94.12,95.89,90.12
$630-644,78.06,81.43,71.40,83.61,60.29
$645-675,77.05,81.01,66.23,81.11,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [77]:
# Creating data bin based on the given table for School size
#
size_bins = [0, 1000, 2000, 5000]

group_names = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

# Slicing data based on the bin and group_names
#
combined_data['size_bins'] = pd.cut(combined_data['size'], size_bins, labels = group_names)


#group by spending
by_size = combined_data.groupby('size_bins')

#
# Calculation of: Average Math, Reading scores
#
average_math_scores_size = by_size['math_score'].mean()
average_reading_scores_size = by_size['math_score'].mean()

# Calcualte percent of students passing in math based on school size 
#
total_students_passing_in_math_size = combined_data[combined_data['math_score'] >= 70].groupby('size_bins')['Student ID'].count()
rate_of_students_passing_math_size = (total_students_passing_in_math)/by_size['Student ID'].count()
percent_students_passing_math_size = rate_of_students_passing_math*100

# Calcualte percent of students passing in reading based on school size 
#
total_students_passing_reading_size = combined_data[combined_data['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()
rate_of_students_passing_reading_size = total_students_passing_reading/by_size['Student ID'].count()
percent_students_passing_reading_size = rate_of_students_passing_reading*100

# Calcualte overall percent of students passing both in math and reading based on school size 
#
total_students_overall_passing_size = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()
overall_students_passing_rate_size = (total_students_overall_passing_size)/by_size['Student ID'].count()
percent_overall_students_passing_size = overall_students_passing_rate_size*100
            
# 
# 
scores_by_school_size_df = pd.DataFrame({
    "Average Math Score": average_math_scores_size,
    "Average Reading Score": average_reading_scores_size,
    '% Passing Math': percent_students_passing_math_size,
    '% Passing Reading': percent_students_passing_reading_size,
    '% Overall Passing': percent_overall_students_passing_size
            
})
            
#
# Formatting columns based on table that is given
#
scores_by_school_size_df = scores_by_school_size_df[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]


# Format and display data as required
#
scores_by_school_size_df.style.format({'Average Math Score': '{:.2f}', 
                              'Average Reading Score': '{:.2f}', 
                              '% Passing Math': '{:.2f}', 
                              '% Passing Reading':'{:.2f}', 
                              'Overall Passing': '{:.2f}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
$585-629,nan,nan,95.89,nan,nan
$630-644,nan,nan,83.61,nan,nan
$645-675,nan,nan,81.11,nan,nan
<$584,nan,nan,96.69,nan,nan
Large(2000-5000),77.48,77.48,nan,82.13,56.574046
Medium(1000-2000),83.37,83.37,nan,96.77,90.624267
Small(<1000),83.83,83.83,nan,96.04,90.136789


## Scores by School Type

* Perform the same operations as above, based on school type

In [78]:
# Perform groupby school "type"
#
school_type_group = combined_data.groupby("type")

#
# Calculation of: Average Math, Reading scores
#
average_math_score_sc_type = school_type_group['math_score'].mean()
average_reading_score_sc_type = school_type_group['math_score'].mean()

# Calculations: 
# 
total_students_passing_math = combined_data[combined_data['math_score'] >= 70].groupby('type')['Student ID'].count()
percent_students_passing_math_sc_type = (total_students_passing_math*100)/school_type_group['Student ID'].count()

total_students_passing_reading = combined_data[combined_data['reading_score'] >= 70].groupby('type')['Student ID'].count()
percent_studets_passing_reading_sc_type = (total_students_passing_reading*100)/school_type_group['Student ID'].count()

total_students_overall_passing_sc_type = combined_data[(combined_data['reading_score'] >= 70) & (combined_data['math_score'] >= 70)].groupby('type')['Student ID'].count()
percent_overall_passing_type = (total_students_overall_passing_sc_type*100)/school_type_group['Student ID'].count()


# Creating dataframe for the above calc too be displayed 
#
scores_by_school_type = pd.DataFrame({
    "Average Math Score": average_math_score_sc_type,
    "Average Reading Score": average_reading_score_sc_type,
    '% Passing Math': percent_students_passing_math_sc_type,
    '% Passing Reading': percent_studets_passing_reading_sc_type,
    '% Overall Passing': percent_overall_passing_type})
    
# Fromat data columns as per the example 
#
scores_by_school_type = scores_by_school_type[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing'
]]


# scores_by_school_type.index.name = "Type of School"


#
# Format and display data 
#
scores_by_school_type.style.format({'Average Math Score': '{:.6f}', 
                              'Average Reading Score': '{:.6f}', 
                              '% Passing Math': '{:.6}', 
                              '% Passing Reading':'{:.6}', 
                              '% Overall Passing': '{:.6}'})


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.406183,93.7018,96.6459,90.5609
District,76.987026,76.987026,66.5184,80.9052,53.6959
